In [1]:
from utils import *

# FEATURE ENGINEERING

In [2]:
df=pd.read_csv("E:\Backup\Data\processed_data.csv")

<>:1: SyntaxWarning: invalid escape sequence '\B'
<>:1: SyntaxWarning: invalid escape sequence '\B'
C:\Users\Admin\AppData\Local\Temp\ipykernel_25952\1115574706.py:1: SyntaxWarning: invalid escape sequence '\B'
  df=pd.read_csv("E:\Backup\Data\processed_data.csv")


In [3]:
df.columns

Index(['ITEMNAME', 'PATTERN', 'COLOR', 'SUPPCITY', 'MONTH', 'NETSALESRATE',
       'PROFITPER', 'SALESQTY', 'SALESAMOUNT'],
      dtype='object')

**COMPUTING UNIT SALES**

In [4]:
#Computing unit sales
df['SALESAMOUNT'] = df['SALESAMOUNT'].str.replace(',', '')  # remove commas inside strings
df['SALESAMOUNT'] = df['SALESAMOUNT'].astype(float)  # remove commas
df['SALESAMOUNT'] = df['SALESAMOUNT'].replace('.', '')  # remove decimal point
df['UNIT_SALE_VALUE'] = df['SALESAMOUNT']/ df['SALESQTY']

#We see there are null values due to this feature engineering
print('Number of null values in the new column:',df['UNIT_SALE_VALUE'].isnull().sum())

#So lets fill them with mean
df['UNIT_SALE_VALUE']=df['UNIT_SALE_VALUE'].fillna(df['UNIT_SALE_VALUE'].mean())
print('Number of null values in the new column after mean imputation:',df['UNIT_SALE_VALUE'].isnull().sum())

#Lets replace the abnormally huge values with mean
df['UNIT_SALE_VALUE'].replace([np.inf, -np.inf],df['UNIT_SALE_VALUE'].mean() , inplace=True)

Number of null values in the new column: 73
Number of null values in the new column after mean imputation: 0


C:\Users\Admin\AppData\Local\Temp\ipykernel_25952\2387654714.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['UNIT_SALE_VALUE'].replace([np.inf, -np.inf],df['UNIT_SALE_VALUE'].mean() , inplace=True)


**Absolute profit**

Converts profit percent to actual profit amount

In [5]:
#Absolute profit
df['PROFIT'] = (df['PROFITPER'] / 100) * df['NETSALESRATE']


**Discounted or not**

In [6]:
#Discount
df['IS_DISCOUNTED'] = (df['UNIT_SALE_VALUE'] < df['NETSALESRATE']).astype(int)
df['IS_DISCOUNTED']=df['IS_DISCOUNTED'].replace(0,'No Discount')
df['IS_DISCOUNTED']=df['IS_DISCOUNTED'].replace(1,'Discounted')
print(df['IS_DISCOUNTED'])

0       No Discount
1       No Discount
2       No Discount
3       No Discount
4       No Discount
           ...     
6096     Discounted
6097     Discounted
6098    No Discount
6099    No Discount
6100     Discounted
Name: IS_DISCOUNTED, Length: 6101, dtype: object


**Item Frequency**

In [7]:

item_counts = df['ITEMNAME'].value_counts()
df['ITEM_FREQ'] = df['ITEMNAME'].map(item_counts)


Since profit is skewed we log normalise it

In [8]:

df['LOG_PROFIT'] = np.log1p(df['PROFIT'])
df['LOG_PROFIT']=df['LOG_PROFIT'].fillna(df['LOG_PROFIT'].mean())

c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


Rate log is the natural log transformation of NETSALESRATE using log1p (log(1 + x)).
The purpose is to smooth out skewness in the distribution, reduce the impact of extreme values, and make the variable more suitable for regression models.

In [9]:

df['RATE_QTY'] = df['NETSALESRATE'] * df['SALESQTY']
df['RATE_SALEVALUE_DIFF'] = df['UNIT_SALE_VALUE'] - df['NETSALESRATE']
df['RATE_LOG'] = np.log1p(df['NETSALESRATE'])  #For a  Smooth skew

In [10]:
df.to_csv('Processed&FeatureEngineered.csv')